In [1]:
import tensorflow as tf 
print(tf.__version__)

2.16.1


In [2]:
import keras
keras.__version__

'3.0.5'

In [3]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.stem.porter import PorterStemmer
import re

In [4]:
# PUTTING ALL TEXT FROM DIFF DOCS INTO ONE FILE

In [5]:
def create_train_file(file_paths, output_file):
    with open(output_file, 'w') as outfile:
        for file_path in file_paths:
            with open(file_path, 'r') as infile:
                for line in infile:
                    outfile.write(line)

# List of file paths
file_paths = [".\Training Data\dataset1.txt", '.\Training Data\dataset2_test_output.txt', '.\Training Data\dataset3_small_1.txt']

# Output file
output_file = 'all_data_combined.txt'

# Create train.txt file with concatenated text
create_train_file(file_paths, output_file)


In [6]:
# SENDING THAT TRAIN FILE AS INPUT

In [8]:
train=pd.read_table('all_data_combined.txt', delimiter = ';', header=None, )
val=pd.read_table('val.txt', delimiter = ';', header=None, )
test=pd.read_table('test.txt', delimiter = ';', header=None, )
data = pd.concat([train ,  val , test])
data.columns = ["text", "label"]

# val=pd.read_table('val.txt', delimiter = ';', header=None, )
# test=pd.read_table('test.txt', delimiter = ';', header=None, )
# data = pd.concat([train ,  val , test])

In [9]:
data.shape # previously 20000 data now 452809   (dataset1 + dataset2 = (36000, 2))

(100000, 2)

In [10]:
data.isna().any(axis=1).sum()

0

In [11]:
#text preprocessing
ps = PorterStemmer()

def preprocess(line):
    review = re.sub('[^a-zA-Z]', ' ', line) #not alphabets it replaces with space
    review = review.lower() #lower the text
    review = review.split() #turn string into list of words
    #apply Stemming 
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #delete stop words like I, and ,OR   review = ' '.join(review)
    #turn list into sentences
    return " ".join(review)


In [1]:
data['text']=data['text'].apply(lambda x: preprocess(x))


In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
data['N_label'] = label_encoder.fit_transform(data['label'])


In [ ]:
data['text']

In [ ]:
# Creating the Bag of Words model by applying Countvectorizer -convert textual data to numerical data

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000,ngram_range=(1,3))

data_cv = cv.fit_transform(data['text']).toarray()




In [ ]:
data_cv

In [ ]:
X_train, X_test, y_train, y_test =train_test_split(data_cv, data['N_label'], test_size=0.25, random_state=42)


In [55]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(12, input_shape=(X_train.shape[1],), activation='relu')) # Rectified Linear Unit 
model.add(Dense(8, activation='relu')) # Returns zero if its negativate and returns as it is if value is positive
model.add(Dense(6, activation='softmax')) #used for classification problems
#  For multi-class classification problems where the labels are integers
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=10, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

C:\Users\kumar_lf3uub3\Documents\python\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.7283 - loss: 0.7587
Epoch 2/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9185 - loss: 0.2150
Epoch 3/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9380 - loss: 0.1593
Epoch 4/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - accuracy: 0.9491 - loss: 0.1293
Epoch 5/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - accuracy: 0.9585 - loss: 0.1058
Epoch 6/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - accuracy: 0.9632 - loss: 0.0930
Epoch 7/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9680 - loss: 0.0797
Epoch 8/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 28s 4ms/step - accuracy: 0.9711 - loss: 0.0691
Epoch 9/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - accuracy: 0.9746 - loss: 0.0628
Epoch 10/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9771 - loss: 0.0555
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9804 - loss: 0.0476
Accuracy: 98.14


In [56]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8997 - loss: 0.4511
Accuracy: 90.10


In [57]:
import numpy as np
text='I feel sad'
text=preprocess(text)
array = cv.transform([text]).toarray()
pred = model.predict(array)
a=np.argmax(pred, axis=1) 
label_encoder.inverse_transform(a)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step


'sadness'

# Emotion Detector For Text

In [58]:
def emotion_for_text(text):
    text=preprocess(text)
    array = cv.transform([text]).toarray()
    pred = model.predict(array)
    a=np.argmax(pred, axis=1) #index of the highest probability prediction is extracted 
    return label_encoder.inverse_transform(a)[0]
    
emotion_for_text('I hate you')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


'anger'

# Emotion Detector For File

In [59]:
def emotion_for_file(file_path):
    # Open the text file
    with open(file_path, 'r') as file:
        # Iterate over each line in the file
        for line in file:
            # Preprocess the line
            preprocessed_text = preprocess(line)
            # Transform preprocessed text into numerical representation
            input_array = cv.transform([preprocessed_text]).toarray()
            # Get predictions from the model
            predictions = model.predict(input_array)
            # Convert predictions to labels
            predicted_label_index = np.argmax(predictions, axis=1)
            predicted_label = label_encoder.inverse_transform(predicted_label_index)[0]
            # Print the predicted label for the current line
            print("Input Text:", line.strip())1
            print("Predicted Label:", predicted_label)
            print('Accuracy: %.2f' % (accuracy*100))

# Test the model on a text file
emotion_for_file('sample_text.txt')
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Input Text: hello, I love you
Predicted Label: love
Accuracy: 90.10


# Comparing Accuracy

In [60]:
classes = label_encoder.classes_

# Print all the classes
print("All Classes:")
for cls in classes:
    print(cls)

All Classes:
anger
fear
joy
love
sadness
surprise


In [61]:
# Accuracies for all classes
def accuracy_for_all_classes(file_path):
    with open(file_path, 'r') as file:
        sample_text = file.read()

    # Preprocess the text
    preprocessed_text = preprocess(sample_text)

    # Transform preprocessed text into numerical representation
    input_array = cv.transform([preprocessed_text]).toarray()

    # Get predictions from the model
    predictions = model.predict(input_array)

    # Count occurrences of each class label
    class_counts = np.sum(predictions, axis=0)

    # Calculate percentages
    total_predictions = len(predictions)
    class_percentages = class_counts / total_predictions * 100

    # Get class labels
    classes = label_encoder.classes_

    # Print percentages for all classes
    print("Class Percentages for the Sample Text:")
    for cls, percentage in zip(classes, class_percentages):
        print(f"{cls}: {percentage:.2f}%")


In [62]:
tf.keras.models.save_model(model,'my_model.h5')


In [63]:
import pickle
pickle.dump(label_encoder, open('encoder.pkl', 'wb'))
pickle.dump(cv, open('CountVectorizer.pkl', 'wb'))


# Testing

In [67]:
# Test the model on a text file
emotion_for_file('sample_text.txt')
accuracy_for_all_classes('sample_text.txt')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Input Text: hello, I love you
Predicted Label: love
Accuracy: 90.10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Class Percentages for the Sample Text:
anger: 0.38%
fear: 4.11%
joy: 28.46%
love: 58.98%
sadness: 7.87%
surprise: 0.19%


In [73]:
# Evaluate the model on the test dataset
loss, accuracy = loaded_model.evaluate(X_test, y_test)

print("Loss:", loss)
print("Accuracy:", accuracy)


782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8997 - loss: 0.4511
Loss: 0.44759538769721985
Accuracy: 0.9009600281715393


# Loading The Model

In [71]:
print(loaded_model.summary()) # before loading

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 12)                  │          60,012 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 8)                   │             104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 6)                   │              54 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 60,170 (235.04 KB)

 Trainable params: 60,170 (235.04 KB)

 Non-trainable params: 0 (0.00 B)

None


In [1]:
import tensorflow as tf
import pickle

# loading
model = tf.keras.models.load_model('my_model.h5')

# Recompile the model with the same settings
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Save the model again
# tf.keras.models.save_model(model, 'my_model_updated.h5')
# Print model summary
model.summary()

# Alternatively, you can inspect the model architecture
print(model.layers)

# Load the label encoder
with open('encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Load the CountVectorizer
with open('CountVectorizer.pkl', 'rb') as f:
    cv = pickle.load(f)

    #model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 12)                  │          60,012 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 8)                   │             104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 6)                   │              54 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 60,170 (235.04 KB)

 Trainable params: 60,170 (235.04 KB)

 Non-trainable params: 0 (0.00 B)

[<Dense name=dense_3, built=True>, <Dense name=dense_4, built=True>, <Dense name=dense_5, built=True>]


# Evaluation With Test Data And With 'Model'

In [9]:
# val dataset

In [7]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import load_model
import pickle

# Load the test data
test_data = pd.read_table('val.txt', delimiter=';', header=None)
test_data.columns = ["text", "actual_emotion"]

# Load the encoder
with open("encoder.pkl", "rb") as encoder_file:
    loaded_encoder = pickle.load(encoder_file)

# Load the CountVectorizer
with open("CountVectorizer.pkl", "rb") as cv_file:
    loaded_cv = pickle.load(cv_file)

# Preprocess the test data
def preprocess_test_data(text):
    ps = PorterStemmer()
    review = re.sub('[^a-zA-Z]', ' ', text) # leave only characters from a to z
    review = review.lower() # lower the text
    review = review.split() # turn string into list of words
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # delete stop words like I, and, OR
    return " ".join(review)

test_data['text'] = test_data['text'].apply(preprocess_test_data)

# Transform text data into numerical representation using CountVectorizer
X_test = loaded_cv.transform(test_data['text']).toarray()

# Load the trained model
loaded_model = load_model("my_model.h5")

# Make predictions
predictions = loaded_model.predict(X_test)

# Convert predictions to labels
predicted_label_indices = np.argmax(predictions, axis=1)
predicted_labels = loaded_encoder.inverse_transform(predicted_label_indices)

# Calculate accuracy
actual_labels = test_data['actual_emotion'].values
accuracy = np.mean(predicted_labels == actual_labels)

# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'Text': test_data['text'],
    'Original': test_data['actual_emotion'],
    'Predicted': predicted_labels
})

# Print the DataFrame
print(results_df)
# Print accuracy
print("Accuracy:", accuracy)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
                                                   Text Original Predicted
0                  im feel quit sad sorri ill snap soon  sadness   sadness
1     feel like still look blank canva blank piec paper  sadness   sadness
2                               feel like faith servant     love      love
3                                      feel cranki blue    anger     anger
4                                     treat feel festiv      joy       joy
...                                                 ...      ...       ...
1995  im ssa examin tomorrow morn im quit well prepa...  sadness   sadness
1996  constantli worri fight natur push limit inner ...      joy       joy
1997                feel import share info experi thing      joy       joy
1998  truli feel passion enough someth stay true suc...      joy       joy
1999   feel like wanna buy cute make see onlin even one      joy       joy

[2000 rows x 3 columns]
Accuracy: 0.958


In [ ]:
# test dataset

In [8]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import load_model
import pickle

# Load the test data
test_data = pd.read_table('test.txt', delimiter=';', header=None)
test_data.columns = ["text", "actual_emotion"]

# Load the encoder
with open("encoder.pkl", "rb") as encoder_file:
    loaded_encoder = pickle.load(encoder_file)

# Load the CountVectorizer
with open("CountVectorizer.pkl", "rb") as cv_file:
    loaded_cv = pickle.load(cv_file)

# Preprocess the test data
def preprocess_test_data(text):
    ps = PorterStemmer()
    review = re.sub('[^a-zA-Z]', ' ', text) # leave only characters from a to z
    review = review.lower() # lower the text
    review = review.split() # turn string into list of words
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # delete stop words like I, and, OR
    return " ".join(review)

test_data['text'] = test_data['text'].apply(preprocess_test_data)

# Transform text data into numerical representation using CountVectorizer
X_test = loaded_cv.transform(test_data['text']).toarray()

# Load the trained model
loaded_model = load_model("my_model.h5")

# Make predictions
predictions = loaded_model.predict(X_test)

# Convert predictions to labels
predicted_label_indices = np.argmax(predictions, axis=1)
predicted_labels = loaded_encoder.inverse_transform(predicted_label_indices)

# Calculate accuracy
actual_labels = test_data['actual_emotion'].values
accuracy = np.mean(predicted_labels == actual_labels)

# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'Text': test_data['text'],
    'Original': test_data['actual_emotion'],
    'Predicted': predicted_labels
})

# Print the DataFrame
print(results_df)
# Print accuracy
print("Accuracy:", accuracy)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
                                                   Text Original Predicted
0                 im feel rather rotten im ambiti right  sadness   sadness
1                             im updat blog feel shitti  sadness   sadness
2            never make separ ever want feel like asham  sadness   sadness
3     left bouquet red yellow tulip arm feel slightl...      joy       joy
4                                   feel littl vain one  sadness   sadness
...                                                 ...      ...       ...
1995  keep feel like someon unkind wrong think get b...    anger     anger
1996            im feel littl cranki neg doctor appoint    anger     anger
1997              feel use peopl give great feel achiev      joy       joy
1998  im feel comfort derbi feel though start step s...      joy       joy
1999  feel weird meet w peopl text like dont talk fa...     fear      fear

[2000 rows x 3 columns]
Accuracy: 0.947
